In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.9.1+cu126
True


In [ ]:
#pip install pytorch-lightning


  Using cached pytorch_lightning-2.5.6-py3-none-any.whl.metadata (20 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached torchmetrics-1.8.2-py3-none-any.whl.metadata (22 kB)
  Using cached lightning_utilities-0.15.2-py3-none-any.whl.metadata (5.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.8.0-cp310-cp310-win_amd64.whl.metadata (21 kB)
  Using cached multidict-6.7.0-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
Using cached pytorch_lightning-2.5.6-py3-none-any.whl (831 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached lightning_utilities-0.15.2-py3-none-

In [ ]:
#pip install pytorch-metric-learning


  Using cached pytorch_metric_learning-2.9.0-py3-none-any.whl.metadata (18 kB)
  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   --------------------- ------------------ 4.7/8.9 MB 25.9 MB/s eta 0:00:01
   -------------------------------------- - 8.7/8.9 MB 25.6 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 22.1 MB/s  0:00:00
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ------ --------------------------------- 6.3/41.3 MB 32.1 MB/s eta 0:00:02
   ------------ --------------------------- 12.8/41.3 MB 32.2 MB/s eta 0:00:01
   ------------------- -------------------- 19.7/41.3 MB 32.7 MB/s eta 0:00:01
   ------------------------- ---------

In [ ]:
#pip install xgboost


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   --- ------------------------------------ 6.0/72.0 MB 30.7 MB/s eta 0:00:03
   ------ --------------------------------- 12.6/72.0 MB 31.5 MB/s eta 0:00:02
   ---------- ----------------------------- 18.6/72.0 MB 30.9 MB/s eta 0:00:02
   ------------- -------------------------- 24.9/72.0 MB 30.9 MB/s eta 0:00:02
   ------------------ --------------------- 32.5/72.0 MB 32.2 MB/s eta 0:00:02
   ---------------------- ----------------- 39.8/72.0 MB 33.3 MB/s eta 0:00:01
   -------------------------- ------------- 47.7/72.0 MB 33.7 MB/s eta 0:00:01
   ------------------------------ --------- 55.6/72.0 MB 34.4 MB/s eta 0:00:01
   ----------------------------------- ---- 64.2/72.0 MB 35.3 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 35.8 MB/s eta 0:00:01
   ---------------------------------------  71.8/72.0 MB 35.8 MB/s eta 0:00:01
   ---------------------------------------- 72.0/72.0 MB 29.8 

In [ ]:
# ================================================================
# NOTEBOOK: Siamese Triplet + FC + XGBoost + Embedding visuals per epoch
# ================================================================
# 0) IMPORTS
# ================================================================
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import pytorch_lightning as pl
from pytorch_metric_learning.losses import TripletMarginLoss
from pytorch_metric_learning.miners import TripletMarginMiner

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Ajustes de visualización
sns.set()
plt.rcParams["figure.figsize"] = (9,6)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
# ================================================================
# 1) DATA TRANSFORMS + DATASET
# ================================================================
DATASET_ROOT = r"D:\DEEp\deep\dataset_split"

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(DATASET_ROOT, "train"), transform=train_transform)
val_dataset   = datasets.ImageFolder(os.path.join(DATASET_ROOT, "val"),   transform=test_transform)
test_dataset  = datasets.ImageFolder(os.path.join(DATASET_ROOT, "test"),  transform=test_transform)

class_names = train_dataset.classes
num_classes = len(class_names)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("Clases detectadas:", class_names)

In [ ]:
# ================================================================
# 2) SIAMESE NETWORK
# ================================================================
class SiameseBackbone(nn.Module):
    def __init__(self, embedding_size=128):
        super().__init__()
        self.backbone = models.resnet50(weights="DEFAULT")
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, embedding_size)

    def forward(self, x):
        return self.backbone(x)

siamese = SiameseBackbone().to(device)

In [ ]:
# ================================================================
# 3) PYTORCH LIGHTNING TRAINER PARA TRIPLET LOSS
# ================================================================
class TripletLightning(pl.LightningModule):
    def __init__(self, model, lr=1e-4, margin=0.2):
        super().__init__()
        self.model = model
        self.miner = TripletMarginMiner(margin=margin, type_of_triplets="semihard")
        self.loss_func = TripletMarginLoss(margin=margin)
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        embeddings = self(imgs)
        hard_pairs = self.miner(embeddings, labels)
        loss = self.loss_func(embeddings, labels, hard_pairs)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

lightning_model = TripletLightning(siamese)

trainer = pl.Trainer(max_epochs=5, accelerator=device, log_every_n_steps=20)
trainer.fit(lightning_model, train_loader)

# ⭐ GUARDAR MODELO SIAMÉS
torch.save(siamese.state_dict(), "siamese_triplet.pth")
print("Modelo siamés guardado como siamese_triplet.pth")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\Users\I21326\.conda\envs\VS\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA RTX 6000 Ada Generation') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')`

RuntimeError: Please call `iter(combined_loader)` first.

In [ ]:
# ================================================================
# 4) CLASIFICADOR FC (BACKBONE CONGELADO)
# ================================================================
class Classifier(nn.Module):
    def __init__(self, backbone, emb_size, num_classes):
        super().__init__()
        for p in backbone.parameters():
            p.requires_grad = False

        self.backbone = backbone
        self.fc = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        with torch.no_grad():
            emb = self.backbone(x)
        return self.fc(emb)

classifier = Classifier(siamese, 128, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.fc.parameters(), lr=1e-3)

for epoch in range(8):
    classifier.train()
    running_loss = 0
    for img, lbl in train_loader:
        img, lbl = img.to(device), lbl.to(device)

        optimizer.zero_grad()
        out = classifier(img)
        loss = criterion(out, lbl)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")

# ⭐ GUARDAR CLASIFICADOR FC
torch.save(classifier.state_dict(), "classifier_fc.pth")
print("Clasificador FC guardado como classifier_fc.pth")

In [ ]:
# ================================================================
# 5) EVALUACIÓN DEL CLASIFICADOR FC
# ================================================================
def evaluate(model, loader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            p = out.argmax(1).cpu().numpy()
            preds.extend(p)
            labels.extend(y.numpy())

    acc = accuracy_score(labels, preds)
    f1  = f1_score(labels, preds, average="macro")
    return acc, f1, labels, preds

acc, f1, y_true, y_pred = evaluate(classifier, test_loader)

print("Accuracy FC:", acc)
print("F1-score FC:", f1)


In [ ]:
# ================================================================
# 6) EXTRAER EMBEDDINGS (PARA XGBOOST)
# ================================================================
def extract_embeddings(model, loader):
    model.eval()
    all_emb, all_lbl = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            emb = model(x).cpu().numpy()
            all_emb.append(emb)
            all_lbl.append(y.numpy())

    return np.vstack(all_emb), np.hstack(all_lbl)

train_emb, train_lbl = extract_embeddings(siamese, train_loader)
test_emb,  test_lbl  = extract_embeddings(siamese, test_loader)

print("Embeddings shape:", train_emb.shape)

In [ ]:
# ================================================================
# 6.1) VISUALIZAR EMBEDDINGS EN 2D
# ================================================================
def plot_embeddings(embeddings, labels, method="pca", title="Embeddings 2D"):
    """
    embeddings: np.array (N, embedding_dim)
    labels: np.array (N,)
    method: "pca" o "tsne"
    """
    if method.lower() == "tsne":
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
    else:
        reducer = PCA(n_components=2)
        
    emb_2d = reducer.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 8))
    num_classes = len(np.unique(labels))
    for c in range(num_classes):
        idx = labels == c
        plt.scatter(emb_2d[idx, 0], emb_2d[idx, 1], label=str(c), alpha=0.6, s=15)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(title)
    plt.xlabel("Dim 1")
    plt.ylabel("Dim 2")
    plt.tight_layout()
    plt.show()

# Visualizar embeddings de entrenamiento
plot_embeddings(train_emb, train_lbl, method="pca", title="Embeddings PCA - Train")

# Visualizar embeddings de test
plot_embeddings(test_emb, test_lbl, method="pca", title="Embeddings PCA - Test")

In [ ]:
# ================================================================
# 7) XGBOOST PARA CLASIFICAR EMBEDDINGS
# ================================================================
xgb = XGBClassifier(
    n_estimators=400,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric='mlogloss'
)

xgb.fit(train_emb, train_lbl)
xgb_preds = xgb.predict(test_emb)

acc_xgb = accuracy_score(test_lbl, xgb_preds)
f1_xgb  = f1_score(test_lbl, xgb_preds, average="macro")

print("🎯 XGBoost Accuracy:", acc_xgb)
print("🔥 XGBoost F1-score:", f1_xgb)

In [ ]:
# ================================================================
# 8) MATRIZ DE CONFUSIÓN XGBOOST
# ================================================================
cm = confusion_matrix(test_lbl, xgb_preds)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap="Greens", xticklabels=class_names, yticklabels=class_names)
plt.title("Matriz de Confusión - XGBoost")
plt.xlabel("Predicción")
plt.ylabel("Verdad")
plt.show()

In [ ]:
# ================================================================
# 9) GUARDAR MODELOS
# ================================================================
torch.save(siamese.state_dict(), "siamese_triplet_backbone.pth")
torch.save(classifier.state_dict(), "classifier_fc_triplet.pth")
xgb.save_model("xgboost_triplet.json")
print("✅ Modelos guardados en la carpeta del notebook")